In [ ]:
from glob import glob
import os
import itertools
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import ImageGrid, make_axes_locatable, host_subplot
import matplotlib.ticker as ticker
from matplotlib.collections import Collection, LineCollection
from matplotlib.artist import allow_rasterization
from matplotlib.font_manager import FontProperties
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.dates as md
import matplotlib.ticker as ticker
from metpy.plots import ctables
#from pyPIPS import disdrometer_module as dis
from pyPIPS import pips_io as pipsio
from pyPIPS import timemodule as tm
from pyPIPS.utils import mtokm, getTimeWindow, interp_along_1D
from pyPIPS import plotmodule as pm
from pyPIPS import PIPS as pips
from pyPIPS import fmcw
from itertools import cycle
import netCDF4 as netcdf
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
#from skimage.restoration import unwrap_phase
#import skimage.feature as SF
import numpy.ma as ma
from scipy import ndimage, interpolate
import pyart
import cartopy.crs as ccrs
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Read in PIPS data
PIPS_data_dir = '/Users/terrell8/sshfs_mounts/depot/data/Projects/VORTEXSE/obsdata/full_PIPS_dataset/'
PIPS_filename = 'parsivel_combined_FMCW_2017_032717_PIPS2A_60s.nc'
PIPS_filepath = os.path.join(PIPS_data_dir, PIPS_filename)
PIPS_ds = xr.open_dataset(PIPS_filepath)
print(PIPS_ds)

In [ ]:
radar_endtimestamp = '201703272359'

radar_startdatetime = datetime.strptime(radar_starttimestamp, '%Y%m%d%H%M')
radar_enddatetime = datetime.strptime(radar_endtimestamp, '%Y%m%d%H%M')

radar_basedir = \
level2_dir = os.path.join(radar_basedir, 'level2')
level2_files = sorted(glob(level2_dir + '/*.ar2v'))

level2_files_to_read = []
for level2_file in level2_files:
    radar_file_timestamp = os.path.basename(level2_file)[4:19]
    radar_file_datetime = datetime.strptime(radar_file_timestamp, '%Y%m%d_%H%M%S')
    if radar_file_datetime >= radar_startdatetime and radar_file_datetime <= radar_enddatetime:
        level2_files_to_read.append(level2_file)
        print(os.path.basename(level2_file))

gridded_output_dir = os.path.join(radar_basedir, 'gridded_new')
if not os.path.exists(gridded_output_dir):
    os.makedirs(gridded_output_dir)

In [ ]:
# Set up grid limits and spacing

xmin = -123000.
xmax = 123000.
dx = 1000.
nx = int((xmax - xmin) / dx) + 1

ymin = -123000.
ymax = 123000.
dy = 1000.
ny = int((ymax - ymin) / dy) + 1

zmin = 500.
zmax = 4500.
dz = 100.
nz = int((zmax - zmin) / dz) + 1
print(nz)

In [ ]:
# Test first radar file

print(level2_files_to_read[0])

radar = pyart.io.read_nexrad_archive(level2_files_to_read[0])

grid = pyart.map.grid_from_radars(
    (radar,),
    grid_shape=(nz, ny, nx),
    grid_limits=((zmin, zmax), (ymin, ymax), (xmin, xmax)),
    fields=['differential_reflectivity', 'reflectivity', 'cross_correlation_ratio'],
    weighting_function='Barnes2')

grid_xr = grid.to_xarray()
print(grid_xr)

In [ ]:
display = pyart.graph.GridMapDisplay(grid)

# create the plot
projection = ccrs.PlateCarree()
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection=projection)

#ax.imshow(grid.fields['REF']['data'][0], origin='lower')
display.plot_grid('reflectivity', 0, vmin=0, vmax=55, projection=projection, cmap="pyart_HomeyerRainbow", ax=ax)
#plt.xlim(-86.7, -85.3)
#plt.ylim(34.0, 36.0)
#start_time = '2017-04-30T20:45'
#end_time = '2017-04-30T20:45'
#plt.title('HTX Time {}'.format(start_time))

# Read in the file, create a RadarMapDisplay object
# fmcw_data_dir = '/Users/terrell8/sshfs_mounts/depot/data/Projects/VORTEXSE/obsdata/2017/FMCW/0430'

# Indicate the radar location with a point
FMCW_lat = 34.6872
FMCW_lon = -86.005
ax.plot([FMCW_lon], [FMCW_lat], color='black', marker='o', transform=ccrs.PlateCarree())


In [ ]:
# Create gridded radar objects and dump to disk
for i, f in enumerate(level2_files_to_read):
    print("On file {:d} of {:d} ({:.2f}%)".format(i+1, len(level2_files), (i+1)/len(level2_files)*100.))
    print("Reading {}".format(os.path.basename(f)))
    radar = pyart.io.read_nexrad_archive(f)
    output_file = os.path.basename(f)[:-5]+"_gridded.nc"
    output_path = os.path.join(gridded_output_dir, output_file)
    
    print("Mapping to grid!")
    grid = pyart.map.grid_from_radars(
        (radar,),
        grid_shape=(nz, ny, nx),
        grid_limits=((zmin, zmax), (ymin, ymax), (xmin, xmax)),
        fields=['differential_reflectivity', 'reflectivity', 'cross_correlation_ratio'],
        weighting_function='Barnes2')
    print("Writing {}".format(output_file))
    pyart.io.write_grid(output_path, grid, arm_time_variables=True, arm_alt_lat_lon_variables=True, 
                        write_point_x_y_z=True, write_point_lon_lat_alt=True)
